In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pubcrawler.article import Article

In [14]:
from IPython.display import clear_output

In [3]:
import os
import pymongo
from bson.objectid import ObjectId
from collections import defaultdict
from random import sample, seed

In [10]:
import time

## Create a list of files

In [7]:
pmc_path = "/Volumes/Transcend/datasets/pmc/2019-04-03/oa_bulk/"

In [8]:
all_files = []
for (dirpath, dirnames, filenames) in os.walk(pmc_path):
    all_files += [os.path.join(dirpath, filename) for filename in filenames if filename.endswith("xml")]

## Counting coverage of different ID types from a sample of 5000 articles

In [125]:
files = sample(all_files, 500)

In [260]:
from IPython.display import clear_output
class Reporter:
    def __init__(self, interval, total):
        self.start = time.time()
        self.this_time = self.start
        self.interval = interval
        self.total = total
        print(self.start, self.this_time, self.interval, self.total)
    
    def report(self, idx):
        self.last_time = self.this_time
        self.this_time = time.time()
        elapsed = self.this_time - self.start
        mean_time_per = elapsed / idx
        mean_time_left = (self.total - idx) * mean_time_per
        this_time_per = (self.this_time - self.last_time) / self.interval
        this_time_left = (self.total - idx) * this_time_per
        output = ""
        output += "Processed {0} articles ({1:.1f}% done).\n".format(idx, idx/self.total * 100)
        output += "Elapsed: {0:.1f} sec\n".format(elapsed)
        output += "Time Per All {0}: {1:.3f} sec    Est Remaining: {2:.1f} sec\n".format(idx, mean_time_per, mean_time_left)
        output += "Time Per Last {0}: {1:.3f} sec    Est Remaining: {2:.1f} sec\n".format(report_every, this_time_per, this_time_left)
        output += "Est Total (Mean): {0:.1f} sec    Est Total (Last {1}): {2:.1f} sec".format(elapsed + mean_time_left, self.interval, elapsed + this_time_left)
        print(output, flush=True)

In [261]:
id_types = defaultdict(int)
article_types = defaultdict(int)

reporter = Reporter(50, len(files))
for idx, filename in enumerate(files):
    if idx is not 0 and idx % reporter.interval is 0:
        reporter.report(idx)
    with open(filename, "r") as f:
        article = Article(f.read())
    
    ids = article.pub_ids().keys()
    if len(ids) is 0:
        id_types["none"] += 1
    else:
        for item in ids:
            id_types[item] += 1
    
    article_type = article.article_type()
    if article_type:
        article_types[article_type] += 1

clear_output()

1554496751.104486 1554496751.104486 50 10000
Processed 50 articles (0.5% done).
Elapsed: 2.2 sec
Time Per All 50: 0.044 sec    Est Remaining: 435.4 sec
Time Per Last 50: 0.044 sec    Est Remaining: 435.4 sec
Est Total (Mean): 437.6 sec    Est Total (Last 50): 437.6 sec
Processed 100 articles (1.0% done).
Elapsed: 4.5 sec
Time Per All 100: 0.045 sec    Est Remaining: 445.8 sec
Time Per Last 50: 0.046 sec    Est Remaining: 458.4 sec
Est Total (Mean): 450.3 sec    Est Total (Last 50): 462.9 sec
Processed 150 articles (1.5% done).
Elapsed: 6.4 sec
Time Per All 150: 0.043 sec    Est Remaining: 422.2 sec
Time Per Last 50: 0.039 sec    Est Remaining: 379.6 sec
Est Total (Mean): 428.7 sec    Est Total (Last 50): 386.0 sec
Processed 200 articles (2.0% done).
Elapsed: 8.8 sec
Time Per All 200: 0.044 sec    Est Remaining: 432.4 sec
Time Per Last 50: 0.048 sec    Est Remaining: 469.5 sec
Est Total (Mean): 441.2 sec    Est Total (Last 50): 478.3 sec


KeyboardInterrupt: 

In [35]:
id_types

defaultdict(int,
            {'pmc': 5000,
             'publisher-id': 3494,
             'doi': 4347,
             'pmid': 4721,
             'other': 33,
             'pii': 285,
             'pmc-scan': 283,
             'coden': 61,
             'art-access-id': 55,
             'manuscript': 50,
             'publisher-manuscript': 3,
             'sici': 7})

In [36]:
article_types

defaultdict(int,
            {'abstract': 108,
             'research-article': 3615,
             'case-report': 238,
             'product-review': 8,
             'review-article': 338,
             'other': 178,
             'book-review': 76,
             'brief-report': 77,
             'letter': 86,
             'correction': 67,
             'news': 29,
             'editorial': 82,
             'article-commentary': 27,
             'meeting-report': 12,
             'discussion': 11,
             'protocol': 10,
             'addendum': 2,
             'announcement': 4,
             'retraction': 3,
             'introduction': 1,
             'expression-of-concern': 1,
             'methods-article': 8,
             'obituary': 7,
             'rapid-communication': 4,
             'report': 3,
             'reply': 1,
             'systematic-review': 3,
             'data-paper': 1})

It's clear that the PMC ID is the most broadly and consistently applied. Which makes sense! Because we're dealing with PMC articles.

In [200]:
with open(sample(all_files, 1)[0], "r") as f:
    xml = f.read()
    article = Article(xml)
    print(article.article_type())
    if article.soup.body:
        print("BODY")
        print(article.extract_text())
    else:
        print("NO BODY")
        print(article.extract_text())

research-article
BODY



Background

The range of malaria has contracted through a century of economic development and disease control
[1-3]. During an era of renewed interest in elimination and eradication there is a need to better understand and quantify the forces behind this recession. A variety of direct control efforts were likely a major factor behind the global contraction in malaria transmission
[2,3], although these gains were often coincident with rapid economic and social development and land use changes
[4-6]. One major aspect of this development that has been shown to significantly impact malaria transmission is urbanization
[7-9].

The process of urbanization includes physical landscape modification and transformation of environs through demand for resources and improved communications. Moreover, urbanization involves significant socio-economic change; generally improved health, housing and increased wealth
[10,11]. These factors, common to urban areas, cause marked ento

## Load articles into a MongoDB collection

In [131]:
client = pymongo.MongoClient("localhost", 27017)
articles = client.pmc.articles

In [132]:
seed("2019-04-04")
files = sample(all_files, 10000)

In [133]:
articles.drop()
reporter = Reporter(50, len(files))
for idx, file in enumerate(files):
    if idx is not 0 and idx % reporter.interval is 0:
        reporter.report(idx)
    with open(file, "r") as f:
        xml = f.read()
        article = Article(xml)
        articles.insert_one({
            "_id": article.pub_ids().get("pmc"),  # Assuming they all have them
            "xml": xml,
            "extracted_text": article.extract_text()
        })

Processed 9950 articles (99.5% done).
Elapsed: 519.1 sec
Time Per All 9950: 0.052 sec    Est Remaining: 2.6 sec
Time Per Last 50: 0.054 sec    Est Remaining: 2.7 sec
Est Total (Mean): 521.7 sec    Est Total (This): 521.8 sec


In [135]:
articles.find_one()

{'_id': '3904948',
 'xml': '<!DOCTYPE article PUBLIC "-//NLM//DTD JATS (Z39.96) Journal Archiving and Interchange DTD v1.0 20120330//EN" "JATS-archivearticle1.dtd">\n<article xmlns:xlink="http://www.w3.org/1999/xlink" xmlns:mml="http://www.w3.org/1998/Math/MathML" article-type="research-article"><?properties open_access?><front><journal-meta><journal-id journal-id-type="nlm-ta">PLoS One</journal-id><journal-id journal-id-type="iso-abbrev">PLoS ONE</journal-id><journal-id journal-id-type="publisher-id">plos</journal-id><journal-id journal-id-type="pmc">plosone</journal-id><journal-title-group><journal-title>PLoS ONE</journal-title></journal-title-group><issn pub-type="epub">1932-6203</issn><publisher><publisher-name>Public Library of Science</publisher-name><publisher-loc>San Francisco, USA</publisher-loc></publisher></journal-meta><article-meta><article-id pub-id-type="pmid">24489775</article-id><article-id pub-id-type="pmc">3904948</article-id><article-id pub-id-type="publisher-id">PO

Using `collection.update_one()`

In [173]:
for file in files:
    with open(file, "r") as f:
        xml = f.read()
        article = Article(xml)
        articles.update_one(
            filter = {
                "_id": article.pub_ids().get("pmc")
            },
            update = {
                "$set": {
                    "xml": xml,
                    "extracted_text": article.extract_text()
                }
            },
            upsert=True)

## Stuff for Python

TODO: Figure out how to use a boolean query

In [136]:
articles.create_index([("extracted_text", pymongo.TEXT,)])

'extracted_text_text'

## Searching on multiple phrases

### Using sets to record the IDs of matching documents

In [242]:
matching_ids = {}
for term in terms:
    results = articles.find(
        { '$text': { '$search': term } },
        { '_id': '_id', 'score': { '$meta': "textScore" } }
    )
    result_set = {result["_id"] for result in results}
    matching_ids[term] = result_set

In [244]:
matching_any = set.union(*matching_ids.values())
matching_any

{'1247408',
 '1481558',
 '1564427',
 '1779612',
 '1890282',
 '2234391',
 '2453293',
 '2528962',
 '2605341',
 '2606823',
 '2667394',
 '2688363',
 '2689396',
 '2732522',
 '2736310',
 '2738531',
 '2748713',
 '2779852',
 '2789409',
 '2811100',
 '2830968',
 '2882574',
 '2886090',
 '2896373',
 '2908054',
 '2965324',
 '2974751',
 '2995795',
 '3037366',
 '3083667',
 '3111462',
 '3112453',
 '3135997',
 '3163512',
 '3227807',
 '3236786',
 '3249557',
 '3266242',
 '3281017',
 '3287448',
 '3313868',
 '3317851',
 '3320900',
 '3366669',
 '3388975',
 '3426517',
 '3457864',
 '3484178',
 '3494555',
 '3497848',
 '3526392',
 '3536940',
 '3556712',
 '3557901',
 '3562581',
 '3565895',
 '3577466',
 '3580738',
 '3597257',
 '3612753',
 '3638147',
 '3646733',
 '3663739',
 '3670493',
 '3694090',
 '3784500',
 '3808113',
 '3826076',
 '3842058',
 '3846669',
 '3849780',
 '3850638',
 '3853547',
 '3886783',
 '3890265',
 '3922777',
 '3922792',
 '3994642',
 '4014550',
 '4023239',
 '4079231',
 '4109881',
 '4117438',
 '41

### Using a defaultdict() to record which IDs a thing as matched

**USE THIS ONE**

In [248]:
results = defaultdict(list)
for term in terms:
    term_results = articles.find(
        { '$text': { '$search': term } },
        { '_id': '_id', 'score': { '$meta': "textScore" } }
    )
    for r in term_results:
        results[r["_id"]].append(term)
results

defaultdict(list,
            {'3236786': ['"field study"', '"study site"'],
             '3994642': ['"field study"'],
             '4837512': ['"field study"'],
             '5773159': ['"field study"'],
             '4185895': ['"field study"'],
             '5600973': ['"field study"'],
             '3426517': ['"field study"', '"study site"'],
             '5660281': ['"field study"'],
             '3842058': ['"field study"'],
             '3320900': ['"field study"'],
             '5660108': ['"field study"'],
             '3922777': ['"field study"'],
             '4650156': ['"field study"'],
             '4284477': ['"field study"'],
             '5664821': ['"field study"', '"study site"'],
             '4259558': ['"field study"'],
             '4682827': ['"field study"'],
             '4361704': ['"field study"'],
             '4931437': ['"field study"'],
             '3638147': ['"field study"'],
             '3037366': ['"field study"'],
             '6425011': ['"fiel

In [226]:
result_list = [result for result in results]

In [227]:
result_list[0]

{'_id': '3826076', 'score': 1.438532305076512}

In [229]:
result_set = {result["_id"] for result in results}

In [230]:
result_set

{'1247408',
 '1890282',
 '2605341',
 '2667394',
 '2689396',
 '2748713',
 '2789409',
 '2882574',
 '2965324',
 '2974751',
 '3163512',
 '3287448',
 '3313868',
 '3317851',
 '3457864',
 '3526392',
 '3663739',
 '3784500',
 '3826076',
 '3849780',
 '3850638',
 '4023239',
 '4109881',
 '4514805',
 '4919788',
 '5435308',
 '5547140',
 '5738821',
 '5774785',
 '6293958'}

## Basic code for walking with a limit

However, this isn't usable, because we actually need to *randomly* sample the files.

In [86]:
i = 10
for path, dirs, files in os.walk(pmc_path):
    if i <= 0:
        break
    for file in files[:i]:
        if file.endswith("xml"):
            i -= 1
            with open(os.path.join(path, file), "r") as file:
                article = Article(file.read())
            print(article.pub_ids().get("pmid"))
print("Stopped.")

<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
Stopped.


In [85]:
## Counting how many have PMIDs.

In [105]:
foo = dict()

In [107]:
len(foo.keys())

0

In [113]:
id_types = defaultdict(int)

i = 2500
for path, dirs, files in os.walk(pmc_path):
    if i <= 0:
        break
    for file in files[:i]:
        if file.endswith("xml"):
            if i % 25 is 0:
                print(i)
            i -= 1
            with open(os.path.join(path, file), "r") as file:
                article = Article(file.read())
            ids = article.pub_ids().keys()
            if len(ids) is 0:
                id_types["none"] += 1
            else:
                for item in ids:
                    id_types[item] += 1
            
print(id_types)

2500
2475
2450
2425
2400
2375
2350
2325
2300
2275
2250
2225
2200
2175
2150
2125
2100
2075
2050
2025
2000
1975
1950
1925
1900
1875
1850
1825
1800
1775
1750
1725
1700
1675
1650
1625
1600
1575
1550
1525
1500
1475
1450
1425
1400
1375
1350
1325
1300
1275
1250
1225
1200
1175
1150
1125
1100
1075
1050
1025
1000
975
950
925
900
875
850
825
800
775
750
725
700
675
650
625
600
575
550
525
500
475
450
425
400
375
350
325
300
275
250
225
200
175
150
125
100
75
50
25
defaultdict(<class 'int'>, {'pmid': 2484, 'pmc': 2500, 'doi': 2500, 'publisher-id': 2476, 'art-access-id': 4, 'manuscript': 11, 'coden': 2002, 'pii': 1995})
